In [1]:
import pandas as pd
import numpy as np

## 1. Create TMM from TPM (and count) file

### 1.1 Collapse annotation reference
- Need to be the same version as in RNAseq alignment. Otherwise will have a lot of missing genes\
- The version of reference used in alignment is genecode v26

In [15]:
%%bash
# collapse annotation: https://github.com/broadinstitute/gtex-pipeline/tree/master/gene_model
python 00_collapse_annotation.py /data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v44.basic.annotation.gtf.gz \
/data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v44.basic.collapsed.gtf.gz_v2

Parsing GTF: 62700 genes processed
Collapsing transcripts


/data100t1/home/wanying/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/data100t1/home/wanying/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### 1.2 Create sample_participant_id_mapping.tsv
- For eQTL mapping need to map RNAseq IDs to genotype IDs
- For general purpose, do not mapp IDs to avoid missing IDs in the column
    - Samples without genotype data will be labeled as na.1, na.2, etc.

In [4]:
# ID mapping file between genotype ID and RNAseq ID
fn_count = '/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4.rnaseq_count.low_quality_dropped.gct.gz'
df_count = pd.read_csv(fn_count, sep='\t', skiprows=2)
display(df_count.head(2))

fn_id_mapping = '/data100t1/home/wanying/CCHC/doc/samples_IDs/202309_ID_mapping_RNA_genotyping_batch1_4_all_samples.txt'
df_id_mapping = pd.read_csv(fn_id_mapping, sep='\t')
display(df_id_mapping.head(2))


output_fn = '/data100t1/home/wanying/CCHC/eQTL_gtex_pipeline/supporting_files/sample_participant_id_mapping_LABID2genotypeID.tsv'
mask = df_id_mapping['LABID'].isin(df_count.columns)
df_id_mapping[mask][['LABID', 'genotype_ID']].to_csv(output_fn, sep='\t', index=False, header=False)


# ID mapping file keep the original LABID
output_fn = '/data100t1/home/wanying/CCHC/eQTL_gtex_pipeline/supporting_files/sample_participant_id_mapping_LABID2LABID.tsv'
df_id_mapping[mask][['LABID', 'LABID']].to_csv(output_fn, sep='\t', index=False, header=False)


,Name,Description,10Y0001,10Y0002,10Y0003,10Y0004,10Y0009,10Y0012,10Y0014,10Y0016,...,LA0351,LA0352,LA0380,LA0392,LA0399,LD4184,LD4190,LP0048,LP0049,LP0103
0,ENSG00000223972.5,DDX11L1,65,36,60,30,60,23,13,29,...,130,50,28,104,69,23,26,32,29,39
1,ENSG00000227232.5,WASH7P,54,149,122,173,117,77,47,140,...,210,412,68,188,158,202,178,26,196,215


,genotype_ID,RRID,LABID,batch
0,BD0704_BD4704,BD0704,DR2032,5928JB
1,BD0704_BD4704,BD0704,10Y0071,5928JB


### 1.3 Prepare expression file (TMM normalization)

#### (1) Version 1: Convert RNAseq IDs to genotype IDs for eQTL mapping

In [2]:
%%bash
# From GTEx RNAseq pipeline
tpm_gct=/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4.rnaseq_tpm.low_quality_dropped.gct.gz
counts_gct=/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4.rnaseq_count.low_quality_dropped.gct.gz
# annotation_gtf=/data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v44.basic.collapsed.gtf.gz
# Annotation gtf need to be the same version as the reference used in alignment of RNAseq reads
annotation_gtf=/data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v26.collapsed.gtf.gz

# TSV linking sample IDs (columns in expression matrices) to participant IDs (VCF IDs), without header
sample_participant_lookup=/data100t1/home/wanying/CCHC/eQTL_gtex_pipeline/supporting_files/sample_participant_id_mapping_LABID2genotypeID.tsv
prefix=CCHC_batch1_2_3_4_TMM.low_quality_dropped
# vcf_chr_list not used any more

code_path=/data100t1/home/wanying/lab_code/gtex-pipeline/qtl/src
python ${code_path}/eqtl_prepare_expression.py ${tpm_gct} ${counts_gct} ${annotation_gtf} \
    ${sample_participant_lookup} ${prefix} \
    --tpm_threshold 0.1 \
    --count_threshold 6 \
    --sample_frac_threshold 0.2 \
    --normalization_method tmm

Loading expression data
Normalizing data (tmm)
  * 56200 genes in input tables.
  * 26746 genes remain after thresholding.
  * 26701 genes remain after selecting chromosomes.
Writing BED file


/data100t1/home/wanying/lab_code/gtex-pipeline/qtl/src/eqtl_prepare_expression.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bed_df = bed_df.groupby('chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start'))


#### (2) Version 2: Keep original LABIDs

In [3]:
%%bash
# From GTEx RNAseq pipeline
tpm_gct=/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4.rnaseq_tpm.low_quality_dropped.gct.gz
counts_gct=/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4.rnaseq_count.low_quality_dropped.gct.gz
# annotation_gtf=/data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v44.basic.collapsed.gtf.gz
# Annotation gtf need to be the same version as the reference used in alignment of RNAseq reads
annotation_gtf=/data100t1/home/wanying/CCHC/RNAseq_gtex_pipeline/GTEx_ref/gencode.v26.collapsed.gtf.gz


# TSV linking sample IDs (columns in expression matrices) to participant IDs (VCF IDs), without header
sample_participant_lookup=/data100t1/home/wanying/CCHC/eQTL_gtex_pipeline/supporting_files/sample_participant_id_mapping_LABID2LABID.tsv
prefix=CCHC_batch1_2_3_4_TMM.LABID.low_quality_dropped
# vcf_chr_list not used any more

code_path=/data100t1/home/wanying/lab_code/gtex-pipeline/qtl/src
python ${code_path}/eqtl_prepare_expression.py ${tpm_gct} ${counts_gct} ${annotation_gtf} \
    ${sample_participant_lookup} ${prefix} \
    --tpm_threshold 0.1 \
    --count_threshold 6 \
    --sample_frac_threshold 0.2 \
    --normalization_method tmm

Loading expression data
Normalizing data (tmm)
  * 56200 genes in input tables.
  * 26746 genes remain after thresholding.
  * 26701 genes remain after selecting chromosomes.
Writing BED file


/data100t1/home/wanying/lab_code/gtex-pipeline/qtl/src/eqtl_prepare_expression.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bed_df = bed_df.groupby('chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start'))


## 2. Calculate PEERs (TBD)

In [ ]:
%%bash
prefix=CCHC_batch1_2_3_4.low_quality_dropped
num_peer=60
expression=/vgipiper04/CCHC/RNAseq/batch1_2_3_4/GTEx/expression/CCHC_batch1_2_3_4_TMM.low_quality_dropped.expression.bed.gz

Rscript /data100t1/home/wanying/lab_code/gtex-pipeline/qtl/src/run_PEER.R ${expression} \
${prefix} ${num_peer}

## 3. Combine covariates

In [ ]:
# pca_fn=/data100t1/home/wanying/CCHC/doc/genotype_and_pc/202210_CCHC_PCs_id_fixed_1KG_controls_removed.txt

In [ ]:
%%bash
combine_covariates.py ${prefix}.PEER_covariates.txt ${prefix} \
    --genotype_pcs ${genotype_pcs} \
    --add_covariates ${add_covariates}

# parser.add_argument('expression_covariates', help='')
# parser.add_argument('prefix', help='')
# parser.add_argument('--genotype_pcs', default=None, help='Genotype PCs')
# parser.add_argument('--add_covariates', default=[], nargs='+', help='Additional covariates')
# parser.add_argument('-o', '--output_dir', default='.', help='Output directory')